# Create data

In [ ]:
import json
from pycocotools.coco import COCO
from collections import Counter

In [ ]:
jsonfiles = ["/root/data/small_pen_data_collection/body_parts_detection_merged.json"]

In [ ]:
eye_center_data = []
for jfile in jsonfiles:
    coco = COCO(jfile)
    image_ids = coco.getImgIds()
    cats = coco.cats
    print(cats)
    for image_id in image_ids:
        image_data = coco.loadImgs([image_id])[0]
        if "local_path" not in image_data:
            continue

        annotation_ids = coco.getAnnIds(imgIds=[image_data['id']] )   
        headid = coco.getAnnIds(imgIds=[image_data['id']], catIds=[1])
        if len(coco.loadAnns(headid)) > 0:
            head = coco.loadAnns(headid)
            for h in head:
                tmp = {}
                tmp['local_path'] = image_data['local_path'].replace("sotra-small-pen_0", "small-pen-test-site_1")

                tmp['head_bbox'] = h['bbox']
                tmp['head_mask'] = h['segmentation']
                tmp['jfile'] = jfile

                eye_center_data.append(tmp)

In [ ]:
from tqdm import tqdm
import os
import skimage.io as io
import matplotlib.pyplot as plt

In [ ]:
for ec in tqdm(eye_center_data):
    # load the image
    new_path = '/root/data/reidentification/heads/{}.head.jpg'.format(os.path.basename(ec['local_path']))
#    print(new_path)
    if os.path.isfile(new_path):
        continue
    image = io.imread(ec['local_path'])
    
    # x1, y1, width, height = [int(c) for c in ec['head_bbox']]
    x1, y1, x2, y2 = [int(c) for c in ec['head_bbox']]
    # head = image[y1:y1+height, x1:x1+width, :]
    head = image[x1:x2, y1:y2, :]
#     plt.imshow(head)
#     plt.show()
    io.imsave(new_path, head)

In [ ]:
new_path

In [ ]:
ec['head_bbox']

In [ ]:
head

In [ ]:
y1

In [ ]:
y2

# Build generator

In [ ]:
import glob
import json
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import skimage.io as io
from skimage.transform import resize
from pycocotools.coco import COCO
import random
from keras.applications.mobilenet import MobileNet


### list images

In [ ]:
heads = glob.glob('/root/data/reidentification/heads_with_eye/small-pen-test-site/*')
heads += glob.glob('/root/data/reidentification/heads/*')
print(len(heads))

In [ ]:
# len(list(set([os.path.basename(h).split('.')[0] for h in heads])))

In [ ]:
os.path.basename(heads[0]).split('.')[0]

In [ ]:
mappingdic = {}
all_path = glob.glob('/root/data/small_pen_data_collection/1*/*.jpg')
for path in all_path:
    mappingdic[os.path.basename(path).split('.')[0]] = path.split('/')[-2].split('_')[0]

In [ ]:
experiencedic = {}
for head in heads:
    k = os.path.basename(head).split('.')[0]
    exp = mappingdic[k]
    if exp not in experiencedic:
        experiencedic[exp] = []
    experiencedic[exp].append(head)
    

In [ ]:
for (k, v) in experiencedic.items():
    print(k, len(v))

In [ ]:
batch_size = 16
nclasses = len(experiencedic.keys())
print(nclasses)
input_size = (128, 128, 3)
experiences = list(experiencedic.keys())

In [ ]:
def generator(experiencedic, batch_size, input_size):
    
    while True:
        x_batch = np.zeros((batch_size, input_size[0], input_size[1], input_size[2]))
        y_batch = np.zeros((batch_size, nclasses))
        for i in range(batch_size):
            random_exp = np.random.choice(list(experiencedic.keys()))
            random_head = np.random.choice(experiencedic[random_exp])
            img = io.imread(random_head)
            head = resize(img, input_size)
            xscale = input_size[0] / img.shape[0]
            yscale = input_size[1] / img.shape[1]
            x_batch[i, ...] = head
            y_batch[i, experiences.index(random_exp)] = 1
        yield x_batch, y_batch

In [ ]:
train_gen = generator(experiencedic, batch_size, input_size)
# val_gen = generator(experiencedic, batch_size, input_size)

In [ ]:
xb, yb = next(train_gen)
for i in range(batch_size):
    f, ax = plt.subplots(1)
    ax.imshow(xb[i, ...])
    ax.set_title(experiences[np.argmax(yb[i, ...])])
    plt.show()

# LOAD MODEL

In [ ]:
import keras
from keras.applications.mobilenet import MobileNet
from keras import layers
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
alpha = 1.0
dropout=1e-3
classes = nclasses
shape = (1, 1, int(1024 * alpha))

In [ ]:
mnet = MobileNet(input_shape=input_size, weights='imagenet', include_top=False)

In [ ]:
x = layers.GlobalAveragePooling2D()(mnet.output)
x = layers.Reshape(shape, name='reshape_1')(x)
x = layers.Dropout(dropout, name='dropout')(x)
x = layers.Conv2D(classes, (1, 1),
                  padding='same',
                  name='conv_preds')(x)
x = layers.Activation('softmax', name='act_softmax')(x)
x = layers.Reshape((classes,), name='reshape_2')(x)

In [ ]:
model = Model(inputs=mnet.inputs, outputs=[x])

In [ ]:
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["categorical_accuracy", "top_k_categorical_accuracy"])

In [ ]:
history = model.fit_generator(train_gen,
                    steps_per_epoch= len(heads) // batch_size, 
                    epochs=50)